In [1]:
import pandas as pd
import numpy as np
import pdb
from docx import Document
from docx.shared import Mm
from collections import defaultdict

from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt

In [2]:
'''

Import documents and report informations
//this part should be translated to the features on the Graphic User Interface


'''
MERSummary = pd.read_excel('MERSummaryAll (11).xlsx')
EMPS = pd.read_excel('EquipmentMaintenancePerformanceSummary (8).xlsx')
PDTD = pd.read_excel('PlantDowntimeDetail (7).xlsx')
fleet_desc_root = pd.ExcelFile('fleetdesc fixing.xlsx')
fleet_desc_raw = pd.read_excel(fleet_desc_root,'MMA')
fleet_desc_pivot1 = pd.read_excel(fleet_desc_root,'pivotMMA')
fleet_desc_exception = pd.read_excel(fleet_desc_root,'exception')
nPeriod = 'Week 34'
for_only_critical_unit_boolean = False

In [4]:
'''

Take units with grouping on fleet desc columns

'''

fleet_desc_pivot = fleet_desc_pivot1[fleet_desc_pivot1['fleet desc'].notna()]
fleet_desc_pivot

,Description,fleet desc,MTBF Standard,Critical Unit ?
2,Atlas Copco DML SP,Driller,80.0,Yes
3,Atlas Copco XAHS-225DD,Air Compressor,100.0,NaN
4,Atlas Copco XAHS-250 DD,Air Compressor,100.0,NaN
5,Atlas Copco XATS67DD,Air Compressor,100.0,NaN
8,Bomag BW211D-40,Compactor,100.0,NaN
...,...,...,...,...
235,Volvo FH16C,Medium Truck,100.0,NaN
236,Volvo FM330,Medium Truck,100.0,NaN
237,Volvo FM440,Medium Truck,100.0,NaN
238,Volvo FMX330,Medium Truck,100.0,NaN


In [5]:
'''

Choose only critical unit groups.

Group unit, find the mean group's MTBF standard.

'''

fleet_desc_pivot.dropna(inplace=for_only_critical_unit_boolean)
fleet_desc_pivot.groupby('fleet desc').mean('MTBF Standard')

,MTBF Standard
fleet desc,
Air Compressor,100.0
Cat 16M,100.0
Cat 18M,100.0
Cat 330GC,80.0
Cat 740,100.0
Cat 745,100.0
Cat 773D,100.0
Cat 773E,100.0
Cat 777D,100.0


In [6]:
'''


Adjust columns name in order to merge dataframes


'''

fleet_desc_raw.rename(columns={'Unit Number':'Unit'},inplace=True)


fleetdesc = pd.merge(fleet_desc_raw,fleet_desc_pivot,on='Description',how='outer')
fleet_desc_exception.rename(columns={'Unit Number':'Unit'},inplace=True)
fleet_desc_exception

In [10]:
fleetdesc = pd.merge(fleet_desc_raw,fleet_desc_pivot,on="Description",how='inner')
fleetdesc['fleet desc']

0             Genset
1             Genset
2             Genset
3             Genset
4             Genset
           ...      
714    Komatsu HM400
715     Medium Truck
716     Medium Truck
717     Medium Truck
718     Medium Truck
Name: fleet desc, Length: 719, dtype: object

In [13]:
'''
MER Processing

'''

MERSummary.drop(MERSummary.columns[[0,1]],axis=1,inplace=True)
MERSummary.dropna(inplace=True)
MERSummary.reset_index(drop=True,inplace=True)
MERSummary.columns = MERSummary.iloc[0]
MERSummary.drop(0,inplace=True)
indexNames = MERSummary[(MERSummary['Unit']=='OEM Totals')|(MERSummary['Unit']=='Class Total')|(MERSummary['Unit']=='Report Total')].index
MERSummary.drop(indexNames, inplace = True)

In [14]:
MERSummary

,Unit,PWT\n(00),SWT\n(01),IOD-On\n(04),IOD-Off\n(04),EOD-On\n(05),EOD-Off\n(05),PMD\n(08),UMD\n(09),NRT On\n(15),...,PDAM\n%,SWT\n%,RT\n%,U \nof \nAT\n%,Phys\nAvail\n%,Delay\nSMU\n%,Ann \nSMU\nHrs,Ann\nWork\nHrs,OPE\n%,PM\n%
1,VT2210,0.0,79.5,0.0,111.41664,0.0,0.0,0.0,1.08333,0.0,...,0,1,1,0.416412,0.994358,0,3627.188067,3627.188067,0,0
2,VT2212,0.0,2.5,0.0,57.5,0.0,0.0,0.0,132.0,0.0,...,0,1,1,0.041667,0.3125,0,114.0625,114.0625,0.208696,0
3,VT2213,0.0,44.0,0.0,102.00001,0.0,0.0,13.0,32.99999,0.0,...,0,1,1,0.30137,0.760417,0,2007.5,2007.5,0.107843,0.421053
4,VT2214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,192.0,0.0,...,0,0,1,0,0,0,0,0,0,0
6,VT3155,0.0,16.5,0.0,121.0,0.0,0.0,3.16667,51.33333,0.0,...,0,1,1,0.12,0.716146,0,752.8125,752.8125,0,0.058104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,RD3443,0.0,51.06669,0.0,134.10009,0.0,5.0,0.0,1.83333,0.0,...,0,1,1,0.268536,0.990451,0,2329.916396,2329.916396,0.001198,0.083335
520,LO3009,0.0,0.0,0.0,192.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,1,0,0,0,0,0
521,LO3010,0.0,27.0,0.99996,164.00004,0.0,0.0,0.0,0.0,0.0,...,0,1,1,0.140625,1,0.035713,1277.498175,1231.875,0.039899,1
523,LO1694,0.0,66.0,11.00004,114.99996,0.0,0.0,0.0,0.0,0.0,...,0,1,1,0.34375,1,0.142858,3513.126825,3011.25,0,0


In [15]:
'''
EMPS Processing

'''
EMPS.drop(EMPS.columns[[0]],axis=1,inplace=True)
EMPS.dropna(inplace=True)
EMPS.reset_index(drop=True,inplace=True)
EMPS.columns = EMPS.iloc[0]
EMPS.drop(0,inplace=True)
EMPS.rename(columns = {'Unit\nNumber':'Unit'},inplace=True)
indexNames2 = EMPS[(EMPS['Unit']=='OEM Totals')|(EMPS['Unit']=='Class Total')|(EMPS['Unit']=='Report Total')].index
EMPS.drop(indexNames2,inplace=True)

In [16]:
'''Processing PDTD'''
PDTD.dropna(thresh=3,inplace=True) #masih PR ya
PDTD.reset_index(drop=True,inplace=True)
PDTD.columns = PDTD.iloc[0]
PDTD.drop(0,inplace=True)
PDTD.rename(columns = {'Unit No.' : 'Unit'},inplace=True)
PDTD

,Unit,OEM\nDescription,Meter\n(Start),Meter\n(End),Resp.\nGroup,Maintenance\nArea,Event Start Time,Event End Time,Total\nEvent\nMaint.\nDuration,Period\nEvent\nMaint.\nDuration,...,Event\nID,Tradesman,Activity,Reason,System,Part,Failure Mode,Downtime\nOrigin,No Work\nRequired,Is Failure
1,HO0647,JCB 540 170,1.0,1.0,Workshop,Workshop Melawan,2022-07-22 15:15:00,2022-08-09 19:00:00,435.75,192,...,280065,NaN,NaN,017 - Hydraulics,NaN,NaN,NaN,FAIL-Failure,No,Yes
2,DZ1536,CAT D8 R,1.0,1.0,Electrical Trade,Workshop Melawan,2022-08-01 16:00:00,2022-08-09 19:00:00,195,183,...,281862,NaN,09-Unplanned Maintenance (UMD),011 - Engine,NaN,NaN,NaN,NaN,No,No
3,DZ1551,CAT D10 T,1.0,1.0,Field,Field Shop IGS,2022-08-09 04:30:00,2022-08-12 04:30:00,72,2.5,...,283175,NaN,"09-Unplanned Maintenance (UMD),19-Opportune Ma...",000 - PM Service,NaN,NaN,NaN,UREP-Unplanned Repair,No,No
4,DZ1598,CAT D8 R,3888.1,3888.1,Workshop,Workshop TWS,2022-08-04 17:10:00,2022-08-10 08:00:00,134.833333,109.833333,...,282296,NaN,08-Planned Maintenance (PMD),029 - Undercarriage,NaN,NaN,NaN,COMP-Planned Component Replacement,No,No
5,DZ1598,CAT D8 R,3888.1,3888.1,Electrical Trade,Workshop TWS,2022-08-04 17:10:00,2022-08-10 08:00:00,134.833333,109.833333,...,282296,NaN,NaN,019 - LV Electrical,NaN,NaN,NaN,PREP-Planned Repair,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,VT3155,IVE AD240 4x4 MANHAUL TRUCK TRUCK,27096.0,27096.0,Workshop,Workshop TWS,2022-08-08 08:25:00,2022-08-08 08:35:00,0.166667,0.166667,...,283120,NaN,08-Planned Maintenance (PMD),021 - Tyres,Tyres,Tyre,Retorque,PREP-Planned Repair,No,No
1600,VT8198,IVE MP240E31W TRUCK,433170.0,433170.0,Field,Field Shop Pit Melawan,2022-08-03 03:15:00,2022-08-03 11:00:00,7.75,7.75,...,282037,NaN,NaN,011 - Engine,Air,Air Compressor,Failed,FAIL-Failure,No,Yes
1601,LO3010,CAT 980 G,29297.0,29297.0,Workshop,Field Shop IGS,2022-08-04 08:00:00,2022-08-04 14:35:00,6.583333,6.583333,...,282288,"26765 Sudiono,736393 Agus Setia Budi",18-Opportune Maintenance Rostered Planned (POMR),000 - PM Service,NaN,NaN,NaN,SERV-Planned Service,No,No
1602,WD1537,LIN VANTAGE 500 WELDER,7976.5,7976.5,Workshop,Workshop Balikpapan Support Facility,2022-06-30 16:15:00,2022-08-06 07:35:00,879.333333,120.583333,...,276315,NaN,NaN,011 - Engine,Engine,Engine,Engine,FAIL-Failure,No,Yes


In [17]:
PDTD_complete1 = pd.merge(PDTD, fleetdesc, on='Unit', how='left')
PDTD_complete1

,Unit,OEM\nDescription,Meter\n(Start),Meter\n(End),Resp.\nGroup,Maintenance\nArea,Event Start Time,Event End Time,Total\nEvent\nMaint.\nDuration,Period\nEvent\nMaint.\nDuration,...,Permit Type,Insur. Prem. Type,Job Direct Costed,Cat 17,Prev. Leased/ Owner,Functional Location 1,Functional Location 2,fleet desc,MTBF Standard,Critical Unit ?
0,HO0647,JCB 540 170,1.0,1.0,Workshop,Workshop Melawan,2022-07-22 15:15:00,2022-08-09 19:00:00,435.75,192,...,,15,,,,,,Crane,100.0,NaN
1,DZ1536,CAT D8 R,1.0,1.0,Electrical Trade,Workshop Melawan,2022-08-01 16:00:00,2022-08-09 19:00:00,195,183,...,ML,1,,FIN,PL,,,Cat D8R,80.0,Yes
2,DZ1551,CAT D10 T,1.0,1.0,Field,Field Shop IGS,2022-08-09 04:30:00,2022-08-12 04:30:00,72,2.5,...,ML,1,,FIN,PL,,,Cat D10,80.0,Yes
3,DZ1598,CAT D8 R,3888.1,3888.1,Workshop,Workshop TWS,2022-08-04 17:10:00,2022-08-10 08:00:00,134.833333,109.833333,...,ML,1,,FIN,PL,,,Cat D8R,80.0,Yes
4,DZ1598,CAT D8 R,3888.1,3888.1,Electrical Trade,Workshop TWS,2022-08-04 17:10:00,2022-08-10 08:00:00,134.833333,109.833333,...,ML,1,,FIN,PL,,,Cat D8R,80.0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598,VT3155,IVE AD240 4x4 MANHAUL TRUCK TRUCK,27096.0,27096.0,Workshop,Workshop TWS,2022-08-08 08:25:00,2022-08-08 08:35:00,0.166667,0.166667,...,LP,1,,,,,,Medium Truck,100.0,NaN
1599,VT8198,IVE MP240E31W TRUCK,433170.0,433170.0,Field,Field Shop Pit Melawan,2022-08-03 03:15:00,2022-08-03 11:00:00,7.75,7.75,...,LP,1,,,,,,Medium Truck,100.0,NaN
1600,LO3010,CAT 980 G,29297.0,29297.0,Workshop,Field Shop IGS,2022-08-04 08:00:00,2022-08-04 14:35:00,6.583333,6.583333,...,ML,1,,,PL,,,Cat 980 G/H,100.0,NaN
1601,WD1537,LIN VANTAGE 500 WELDER,7976.5,7976.5,Workshop,Workshop Balikpapan Support Facility,2022-06-30 16:15:00,2022-08-06 07:35:00,879.333333,120.583333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
PDTD_complete2 = pd.merge(PDTD, fleet_desc_exception, on='Unit', how='left')
PDTD_complete2

,Unit,OEM\nDescription,Meter\n(Start),Meter\n(End),Resp.\nGroup,Maintenance\nArea,Event Start Time,Event End Time,Total\nEvent\nMaint.\nDuration,Period\nEvent\nMaint.\nDuration,...,Activity,Reason,System,Part,Failure Mode,Downtime\nOrigin,No Work\nRequired,Is Failure,Description,fleet desc
0,HO0647,JCB 540 170,1.0,1.0,Workshop,Workshop Melawan,2022-07-22 15:15:00,2022-08-09 19:00:00,435.75,192,...,NaN,017 - Hydraulics,NaN,NaN,NaN,FAIL-Failure,No,Yes,NaN,NaN
1,DZ1536,CAT D8 R,1.0,1.0,Electrical Trade,Workshop Melawan,2022-08-01 16:00:00,2022-08-09 19:00:00,195,183,...,09-Unplanned Maintenance (UMD),011 - Engine,NaN,NaN,NaN,NaN,No,No,NaN,NaN
2,DZ1551,CAT D10 T,1.0,1.0,Field,Field Shop IGS,2022-08-09 04:30:00,2022-08-12 04:30:00,72,2.5,...,"09-Unplanned Maintenance (UMD),19-Opportune Ma...",000 - PM Service,NaN,NaN,NaN,UREP-Unplanned Repair,No,No,NaN,NaN
3,DZ1598,CAT D8 R,3888.1,3888.1,Workshop,Workshop TWS,2022-08-04 17:10:00,2022-08-10 08:00:00,134.833333,109.833333,...,08-Planned Maintenance (PMD),029 - Undercarriage,NaN,NaN,NaN,COMP-Planned Component Replacement,No,No,NaN,NaN
4,DZ1598,CAT D8 R,3888.1,3888.1,Electrical Trade,Workshop TWS,2022-08-04 17:10:00,2022-08-10 08:00:00,134.833333,109.833333,...,NaN,019 - LV Electrical,NaN,NaN,NaN,PREP-Planned Repair,No,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598,VT3155,IVE AD240 4x4 MANHAUL TRUCK TRUCK,27096.0,27096.0,Workshop,Workshop TWS,2022-08-08 08:25:00,2022-08-08 08:35:00,0.166667,0.166667,...,08-Planned Maintenance (PMD),021 - Tyres,Tyres,Tyre,Retorque,PREP-Planned Repair,No,No,Volvo FM330,Manhaul
1599,VT8198,IVE MP240E31W TRUCK,433170.0,433170.0,Field,Field Shop Pit Melawan,2022-08-03 03:15:00,2022-08-03 11:00:00,7.75,7.75,...,NaN,011 - Engine,Air,Air Compressor,Failed,FAIL-Failure,No,Yes,Iveco MP240E31W,Manhaul
1600,LO3010,CAT 980 G,29297.0,29297.0,Workshop,Field Shop IGS,2022-08-04 08:00:00,2022-08-04 14:35:00,6.583333,6.583333,...,18-Opportune Maintenance Rostered Planned (POMR),000 - PM Service,NaN,NaN,NaN,SERV-Planned Service,No,No,NaN,NaN
1601,WD1537,LIN VANTAGE 500 WELDER,7976.5,7976.5,Workshop,Workshop Balikpapan Support Facility,2022-06-30 16:15:00,2022-08-06 07:35:00,879.333333,120.583333,...,NaN,011 - Engine,Engine,Engine,Engine,FAIL-Failure,No,Yes,NaN,NaN


In [19]:
PDTD_complete2['fleet desc']=PDTD_complete2['fleet desc'].combine_first(PDTD_complete1['fleet desc'])#.fillna(0).astype(str)
#PDTD_complete2.dropna(subset=['fleet desc'],inplace=True)

In [20]:
PDTD_complete2.dropna(subset=['fleet desc'],inplace=True)
PDTD_complete2

,Unit,OEM\nDescription,Meter\n(Start),Meter\n(End),Resp.\nGroup,Maintenance\nArea,Event Start Time,Event End Time,Total\nEvent\nMaint.\nDuration,Period\nEvent\nMaint.\nDuration,...,Activity,Reason,System,Part,Failure Mode,Downtime\nOrigin,No Work\nRequired,Is Failure,Description,fleet desc
0,HO0647,JCB 540 170,1.0,1.0,Workshop,Workshop Melawan,2022-07-22 15:15:00,2022-08-09 19:00:00,435.75,192,...,NaN,017 - Hydraulics,NaN,NaN,NaN,FAIL-Failure,No,Yes,NaN,Crane
1,DZ1536,CAT D8 R,1.0,1.0,Electrical Trade,Workshop Melawan,2022-08-01 16:00:00,2022-08-09 19:00:00,195,183,...,09-Unplanned Maintenance (UMD),011 - Engine,NaN,NaN,NaN,NaN,No,No,NaN,Cat D8R
2,DZ1551,CAT D10 T,1.0,1.0,Field,Field Shop IGS,2022-08-09 04:30:00,2022-08-12 04:30:00,72,2.5,...,"09-Unplanned Maintenance (UMD),19-Opportune Ma...",000 - PM Service,NaN,NaN,NaN,UREP-Unplanned Repair,No,No,NaN,Cat D10
3,DZ1598,CAT D8 R,3888.1,3888.1,Workshop,Workshop TWS,2022-08-04 17:10:00,2022-08-10 08:00:00,134.833333,109.833333,...,08-Planned Maintenance (PMD),029 - Undercarriage,NaN,NaN,NaN,COMP-Planned Component Replacement,No,No,NaN,Cat D8R
4,DZ1598,CAT D8 R,3888.1,3888.1,Electrical Trade,Workshop TWS,2022-08-04 17:10:00,2022-08-10 08:00:00,134.833333,109.833333,...,NaN,019 - LV Electrical,NaN,NaN,NaN,PREP-Planned Repair,No,No,NaN,Cat D8R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597,VT3155,IVE AD240 4x4 MANHAUL TRUCK TRUCK,27294.0,27294.0,Workshop,Workshop TWS,2022-08-07 21:00:00,2022-08-08 03:00:00,6,6,...,09-Unplanned Maintenance (UMD),011 - Engine,Cooling,Hose/Steel Tube,Slight Leak/Weeping,UREP-Unplanned Repair,No,No,Volvo FM330,Manhaul
1598,VT3155,IVE AD240 4x4 MANHAUL TRUCK TRUCK,27096.0,27096.0,Workshop,Workshop TWS,2022-08-08 08:25:00,2022-08-08 08:35:00,0.166667,0.166667,...,08-Planned Maintenance (PMD),021 - Tyres,Tyres,Tyre,Retorque,PREP-Planned Repair,No,No,Volvo FM330,Manhaul
1599,VT8198,IVE MP240E31W TRUCK,433170.0,433170.0,Field,Field Shop Pit Melawan,2022-08-03 03:15:00,2022-08-03 11:00:00,7.75,7.75,...,NaN,011 - Engine,Air,Air Compressor,Failed,FAIL-Failure,No,Yes,Iveco MP240E31W,Manhaul
1600,LO3010,CAT 980 G,29297.0,29297.0,Workshop,Field Shop IGS,2022-08-04 08:00:00,2022-08-04 14:35:00,6.583333,6.583333,...,18-Opportune Maintenance Rostered Planned (POMR),000 - PM Service,NaN,NaN,NaN,SERV-Planned Service,No,No,NaN,Cat 980 G/H


In [21]:
PDTD_complete = PDTD_complete2

In [22]:
PDTD_complete

,Unit,OEM\nDescription,Meter\n(Start),Meter\n(End),Resp.\nGroup,Maintenance\nArea,Event Start Time,Event End Time,Total\nEvent\nMaint.\nDuration,Period\nEvent\nMaint.\nDuration,...,Activity,Reason,System,Part,Failure Mode,Downtime\nOrigin,No Work\nRequired,Is Failure,Description,fleet desc
0,HO0647,JCB 540 170,1.0,1.0,Workshop,Workshop Melawan,2022-07-22 15:15:00,2022-08-09 19:00:00,435.75,192,...,NaN,017 - Hydraulics,NaN,NaN,NaN,FAIL-Failure,No,Yes,NaN,Crane
1,DZ1536,CAT D8 R,1.0,1.0,Electrical Trade,Workshop Melawan,2022-08-01 16:00:00,2022-08-09 19:00:00,195,183,...,09-Unplanned Maintenance (UMD),011 - Engine,NaN,NaN,NaN,NaN,No,No,NaN,Cat D8R
2,DZ1551,CAT D10 T,1.0,1.0,Field,Field Shop IGS,2022-08-09 04:30:00,2022-08-12 04:30:00,72,2.5,...,"09-Unplanned Maintenance (UMD),19-Opportune Ma...",000 - PM Service,NaN,NaN,NaN,UREP-Unplanned Repair,No,No,NaN,Cat D10
3,DZ1598,CAT D8 R,3888.1,3888.1,Workshop,Workshop TWS,2022-08-04 17:10:00,2022-08-10 08:00:00,134.833333,109.833333,...,08-Planned Maintenance (PMD),029 - Undercarriage,NaN,NaN,NaN,COMP-Planned Component Replacement,No,No,NaN,Cat D8R
4,DZ1598,CAT D8 R,3888.1,3888.1,Electrical Trade,Workshop TWS,2022-08-04 17:10:00,2022-08-10 08:00:00,134.833333,109.833333,...,NaN,019 - LV Electrical,NaN,NaN,NaN,PREP-Planned Repair,No,No,NaN,Cat D8R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597,VT3155,IVE AD240 4x4 MANHAUL TRUCK TRUCK,27294.0,27294.0,Workshop,Workshop TWS,2022-08-07 21:00:00,2022-08-08 03:00:00,6,6,...,09-Unplanned Maintenance (UMD),011 - Engine,Cooling,Hose/Steel Tube,Slight Leak/Weeping,UREP-Unplanned Repair,No,No,Volvo FM330,Manhaul
1598,VT3155,IVE AD240 4x4 MANHAUL TRUCK TRUCK,27096.0,27096.0,Workshop,Workshop TWS,2022-08-08 08:25:00,2022-08-08 08:35:00,0.166667,0.166667,...,08-Planned Maintenance (PMD),021 - Tyres,Tyres,Tyre,Retorque,PREP-Planned Repair,No,No,Volvo FM330,Manhaul
1599,VT8198,IVE MP240E31W TRUCK,433170.0,433170.0,Field,Field Shop Pit Melawan,2022-08-03 03:15:00,2022-08-03 11:00:00,7.75,7.75,...,NaN,011 - Engine,Air,Air Compressor,Failed,FAIL-Failure,No,Yes,Iveco MP240E31W,Manhaul
1600,LO3010,CAT 980 G,29297.0,29297.0,Workshop,Field Shop IGS,2022-08-04 08:00:00,2022-08-04 14:35:00,6.583333,6.583333,...,18-Opportune Maintenance Rostered Planned (POMR),000 - PM Service,NaN,NaN,NaN,SERV-Planned Service,No,No,NaN,Cat 980 G/H


In [23]:
PDTD_complete['Period\nEvent\nMaint.\nDuration'] =  PDTD_complete['Period\nEvent\nMaint.\nDuration'].apply(lambda x: round(x,2))

In [24]:
PDTD_complete['Activity'].replace(',','',regex=True,inplace=True)

In [25]:
PDTD_complete.dropna(subset=['fleet desc'],inplace=True)

In [26]:
PDTD_complete['fleet desc']

0             Crane
1           Cat D8R
2           Cat D10
3           Cat D8R
4           Cat D8R
           ...     
1597        Manhaul
1598        Manhaul
1599        Manhaul
1600    Cat 980 G/H
1602         Genset
Name: fleet desc, Length: 1587, dtype: object

In [27]:
'''Combine MER & EMPS'''
Combined = pd.merge(MERSummary,EMPS, on='Unit', how='outer')

In [28]:
Combined

,Unit,PWT\n(00),SWT\n(01),IOD-On\n(04),IOD-Off\n(04),EOD-On\n(05),EOD-Off\n(05),PMD\n(08),UMD\n(09),NRT On\n(15),...,Plant\nDamage %\nPeriod,Total\nEngine On\nPrevious 28 Days,Number of\nFailures\nPrevious 28 Days,MTBF\nPrevious 28 Days,MTTR-F\nPrevious 28 Days,Mech\nAvail\nPrevious 28 Days,Planned\nMaint %\nPrevious 28 Days,Physical\nAvail\nPrevious 28 Days,Opportune\nEfficiency\n% Previous 28 Days,Plant\nDamage %\nPrevious 28 Days
0,VT2210,0.0,79.5,0.0,111.41664,0.0,0.0,0.0,1.08333,0.0,...,0,221.50013,3,73.833377,25.972223,0.866443,0.044843,0.866443,0.008778,0
1,VT2212,0.0,2.5,0.0,57.5,0.0,0.0,0.0,132.0,0.0,...,0,122.00001,2,61.000005,30.916665,0.65253,0.065402,0.65253,0.039231,0
2,VT2213,0.0,44.0,0.0,102.00001,0.0,0.0,13.0,32.99999,0.0,...,0,198.00009,1,198.00009,46.0,0.831349,0.247657,0.831349,0.030833,0
3,VT2214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,192.0,0.0,...,0,0.0,0,0.0,1,0,0,0,0,0
4,VT3155,0.0,16.5,0.0,121.0,0.0,0.0,3.16667,51.33333,0.0,...,0,97.50003,7,13.928576,20.988096,0.707961,0.076174,0.707961,0.001763,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,RD5187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0.0,0,0.0,1,1,0,1,0,0
426,RD3030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,20.30001,1,20.30001,3.0,0.955295,0,0.955295,0,0
427,RD3036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,12.70001,0,12.70001,1,1,1,1,0.000449,0
428,RD3254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,15.40001,1,15.40001,3.0,0.992188,0.052633,0.992188,0.000456,0


In [29]:
Combined1 =  pd.merge(Combined, fleet_desc_exception, on='Unit', how='left')
Combined1['fleet desc'].tolist()

['Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 nan,
 nan,
 nan,
 nan,
 nan,
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [30]:
Combined2 = pd.merge(Combined, fleetdesc, on='Unit', how='left')
Combined1['fleet desc'] = Combined1['fleet desc'].combine_first(Combined2['fleet desc'])#.fillna(0).astype(str)
Combined1.dropna(subset=['fleet desc'],inplace=True)
All_KPI = Combined1
All_KPI

,Unit,PWT\n(00),SWT\n(01),IOD-On\n(04),IOD-Off\n(04),EOD-On\n(05),EOD-Off\n(05),PMD\n(08),UMD\n(09),NRT On\n(15),...,Number of\nFailures\nPrevious 28 Days,MTBF\nPrevious 28 Days,MTTR-F\nPrevious 28 Days,Mech\nAvail\nPrevious 28 Days,Planned\nMaint %\nPrevious 28 Days,Physical\nAvail\nPrevious 28 Days,Opportune\nEfficiency\n% Previous 28 Days,Plant\nDamage %\nPrevious 28 Days,Description,fleet desc
0,VT2210,0.0,79.5,0.0,111.41664,0.0,0.0,0.0,1.08333,0.0,...,3,73.833377,25.972223,0.866443,0.044843,0.866443,0.008778,0,Isuzu NPS75,Manhaul
1,VT2212,0.0,2.5,0.0,57.5,0.0,0.0,0.0,132.0,0.0,...,2,61.000005,30.916665,0.65253,0.065402,0.65253,0.039231,0,Isuzu NPS75,Manhaul
2,VT2213,0.0,44.0,0.0,102.00001,0.0,0.0,13.0,32.99999,0.0,...,1,198.00009,46.0,0.831349,0.247657,0.831349,0.030833,0,Isuzu NPS75,Manhaul
3,VT2214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,192.0,0.0,...,0,0.0,1,0,0,0,0,0,Isuzu NPS75,Manhaul
4,VT3155,0.0,16.5,0.0,121.0,0.0,0.0,3.16667,51.33333,0.0,...,7,13.928576,20.988096,0.707961,0.076174,0.707961,0.001763,0,Volvo FM330,Manhaul
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,LO3026,0.0,17.0,5.00004,169.99996,0.0,0.0,0.0,0.0,0.0,...,1,92.00004,3.33333,0.959325,0.687351,0.959325,0.013598,0,NaN,Cat 980 G/H
426,RD3030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,20.30001,3.0,0.955295,0,0.955295,0,0,Cat 785C,Mud Truck
427,RD3036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,12.70001,1,1,1,1,0.000449,0,Cat 785C,Mud Truck
428,RD3254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,15.40001,3.0,0.992188,0.052633,0.992188,0.000456,0,Cat 785C,Mud Truck


In [31]:
'''Combine fleet desc & combined'''
#All_KPI = pd.merge(Combined2,fleetdesc,on='Unit', how='outer')
#All_KPI.drop(columns=['Check'],inplace=True)
All_KPI.dropna(thresh=24, inplace=True)


'''round selected column'''
All_KPI['PMD\n(08)'] = All_KPI['PMD\n(08)'].apply(lambda x: round(x,2))
All_KPI['UMD\n(09)'] = All_KPI['UMD\n(09)'].apply(lambda x: round(x,2))
All_KPI['Total\nEngine On\nPeriod'] =  All_KPI['Total\nEngine On\nPeriod'].apply(lambda x: round(x,2))
All_KPI['PDAM\n08020\n09020'] =  All_KPI['PDAM\n08020\n09020'].apply(lambda x: round(x,2))


In [32]:
A = All_KPI.groupby(['fleet desc']).count()
A.to_excel('fleet_desc_fleet.xlsx')

In [33]:
All_KPI[All_KPI['fleet desc']=='CAT 18M']

,Unit,PWT\n(00),SWT\n(01),IOD-On\n(04),IOD-Off\n(04),EOD-On\n(05),EOD-Off\n(05),PMD\n(08),UMD\n(09),NRT On\n(15),...,Number of\nFailures\nPrevious 28 Days,MTBF\nPrevious 28 Days,MTTR-F\nPrevious 28 Days,Mech\nAvail\nPrevious 28 Days,Planned\nMaint %\nPrevious 28 Days,Physical\nAvail\nPrevious 28 Days,Opportune\nEfficiency\n% Previous 28 Days,Plant\nDamage %\nPrevious 28 Days,Description,fleet desc


In [34]:
'''Gather all required variables'''
PWT = All_KPI.groupby(['fleet desc'])['PWT\n(00)'].sum() #F
SWT = All_KPI.groupby(['fleet desc'])['SWT\n(01)'].sum() #G
IODOn = All_KPI.groupby(['fleet desc'])['IOD-On\n(04)'].sum() #H
IODOff = All_KPI.groupby(['fleet desc'])['IOD-Off\n(04)'].sum() #I
EODOn = All_KPI.groupby(['fleet desc'])['EOD-On\n(05)'].sum() #J
EODOff = All_KPI.groupby(['fleet desc'])['EOD-Off\n(05)'].sum() #K
PDAM = All_KPI.groupby(['fleet desc'])['PDAM\n08020\n09020'].sum() #T
PMD = All_KPI.groupby(['fleet desc'])['PMD\n(08)'].sum() #L
UMD = All_KPI.groupby(['fleet desc'])['UMD\n(09)'].sum() #M
EngOn = All_KPI.groupby(['fleet desc'])['Total\nEng On\n(SMU Hrs)'].sum() #V
nFail = All_KPI.groupby(['fleet desc'])['Number of\nFailures\nPeriod'].sum() #W
nFailsum = All_KPI.groupby(['fleet desc'])['Number of\nFailures\nPeriod'].sum()
MTBF_period = All_KPI.groupby(['fleet desc'])['MTBF\nPeriod'].sum() #X
MTTRF_period = All_KPI.groupby(['fleet desc'])[['MTTR-F\nPeriod','Number of\nFailures\nPeriod']] #Y
#nFail_times_MTTRF = (nFail*MTTRF_period).groupby(['fleet desc']).sum()

In [35]:
All_KPI['Failure Hours'] = All_KPI['Number of\nFailures\nPeriod']*All_KPI['MTTR-F\nPeriod']
fail_hours = All_KPI.groupby(['fleet desc'])['Failure Hours'].sum()

In [36]:
'''Do the math'''
MA = ((PWT+SWT+IODOn+IODOff+EODOn+EODOff+PDAM)/(PWT+SWT+IODOn+IODOff+EODOn+EODOff+PDAM+PMD+UMD-PDAM).replace({0:np.nan})).fillna(0)
MTBF =((EngOn)/(nFail).replace({0:np.nan})).fillna(EngOn)
MTTR = ((fail_hours)/(nFailsum).replace({0:np.nan})).fillna(fail_hours)

In [37]:
fail_hours

fleet desc
Cat 16M                41.66666
Cat 18M                19.41666
Cat 330GC                     0
Cat 740               267.99999
Cat 745               179.33334
Cat 773D                      0
Cat 773E                   2.25
Cat 777D                   8.75
Cat 777D CoalTruck        59.75
Cat 777D OB truck     141.08334
Cat 777E               48.73333
Cat 785C OB truck     742.16668
Cat 789C                      0
Cat 834                 5.41667
Cat 980 G/H             2.91667
Cat D10               173.66667
Cat D8R               216.58332
Compactor                   7.5
Crane                         0
Driller                 5.66667
Genset                        0
Hitachi EX2500          6.41667
Hitachi EX2600                0
Hitachi EX3600        154.33334
Hitachi EX5500         91.33333
Hitachi ZX200                 0
Hitachi ZX350         109.08333
Komatsu D65P-12        23.08333
Komatsu D85ESS        277.08333
Komatsu HD785-7         2.58333
Komatsu HM400          18.916

In [38]:
All_KPI['fleet desc'].tolist()

['Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Medium Truck',
 'Medium Truck',
 'Medium Truck',
 'Medium Truck',
 'Medium Truck',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Manhaul',
 'Compactor',
 'Compactor',
 'Crane',
 'Crane',
 'Crane',
 'Crane',
 'Crane',
 'Cat 834',
 'Cat 834',
 'Cat D10',
 'Cat D10',
 'Cat D10',
 'Cat D10',
 'Cat D10',
 'Cat D10',
 'Cat D10',
 'Cat D10',
 'Cat D10',
 'Cat D10',
 'Cat D10',
 'Cat D10',
 'Cat D10',
 'Cat D10',
 'Cat D8R',
 'Cat D8R',
 'Cat D8R',
 'Cat D8R',
 'Cat D8R',
 'Cat D8R',
 'Cat D8R',
 'Cat D8R',
 'Cat D8R',
 'Cat D8R',
 'Cat D8R',
 'Cat D8R',
 'Komatsu D65P-12',
 'Komatsu D65P-12',
 'Komatsu D65P-12',
 'Komatsu D85ESS',
 'Komatsu D85ESS',
 'Komatsu D85ESS',
 'Komatsu D85ESS',
 'Komatsu D85ESS',
 'Komatsu 

In [39]:
'''Combined all KPI'''
Anomali_db = pd.merge(MA.to_frame(),MTBF.to_frame(), on='fleet desc', how= 'outer')
complete_anomali = pd.merge(Anomali_db, MTTR.to_frame(), on='fleet desc', how= 'outer')
complete_anomali.rename(columns = {'0_x':'MA','0_y':'MTBF'},inplace=True)
complete_anomali.columns = ['MA','MTBF','MTTR']
'''round by 2 decimals'''
complete_anomali['MTBF'] = complete_anomali['MTBF'].apply(lambda x: round(x,2))
complete_anomali['MTTR'] = complete_anomali['MTTR'].apply(lambda x: round(x,2))
'''drop zeros'''
complete_KPI = complete_anomali.loc[~(complete_anomali==0).all(axis=1)]
#complete_KPI['MA'] = pd.Series(['{0:,.2f}%'.format(val * 100) for val in complete_KPI['MA']], index = complete_KPI.index)
complete_KPI

,MA,MTBF,MTTR
fleet desc,,,
Cat 16M,0.877876,175.86,10.42
Cat 18M,0.824916,197.03,6.47
Cat 330GC,0.993919,500.93,0.00
Cat 740,0.883735,56.09,9.24
Cat 745,0.903276,82.28,10.55
Cat 773D,0.999115,59.85,0.00
Cat 773E,0.731560,73.48,2.25
Cat 777D,0.943359,24.72,2.92
Cat 777D CoalTruck,0.860354,92.31,3.98


In [40]:
fcd = fleet_desc_pivot.groupby('fleet desc').mean('MTBF Standard')
fcd

,MTBF Standard
fleet desc,
Air Compressor,100.0
Cat 16M,100.0
Cat 18M,100.0
Cat 330GC,80.0
Cat 740,100.0
Cat 745,100.0
Cat 773D,100.0
Cat 773E,100.0
Cat 777D,100.0


In [41]:
combined_MTBF_standard = pd.merge(complete_KPI,fleet_desc_pivot.groupby('fleet desc').mean('MTBF Standard'),how='left',on='fleet desc')
#combined_MTBF_standard.set_index('fleet desc')
combined_MTBF_standard['MTBF']

fleet desc
Cat 16M               175.86
Cat 18M               197.03
Cat 330GC             500.93
Cat 740                56.09
Cat 745                82.28
Cat 773D               59.85
Cat 773E               73.48
Cat 777D               24.72
Cat 777D CoalTruck     92.31
Cat 777D OB truck      78.83
Cat 777E              116.01
Cat 785C OB truck      82.02
Cat 789C                0.00
Cat 834               201.95
Cat 980 G/H            92.41
Cat D10                84.74
Cat D8R                40.63
Compactor              33.68
Crane                 128.00
Driller               160.72
Genset                600.00
Hitachi EX2500        102.47
Hitachi EX2600        121.10
Hitachi EX3600         55.22
Hitachi EX5500         22.72
Hitachi ZX200         227.43
Hitachi ZX350         165.59
Komatsu D65P-12        87.55
Komatsu D85ESS         51.25
Komatsu HD785-7        71.23
Komatsu HM400         151.62
Komatsu PC200          57.69
Komatsu PC300          27.09
Komatsu PC400          24.40
Lie

In [65]:
red_MTBF_this_week1 = combined_MTBF_standard[combined_MTBF_standard['MTBF']<combined_MTBF_standard['MTBF Standard']]#.dropna().set_index('fleet desc')['MTBF']
red_MTBF_this_week = red_MTBF_this_week1['MTBF']
red_MTBF_this_week.index.tolist()

['Cat 740',
 'Cat 745',
 'Cat 773D',
 'Cat 773E',
 'Cat 777D',
 'Cat 789C',
 'Cat 980 G/H',
 'Cat D8R',
 'Compactor',
 'Hitachi EX3600',
 'Hitachi EX5500',
 'Komatsu D85ESS',
 'Komatsu HD785-7',
 'Komatsu PC200',
 'Komatsu PC300',
 'Komatsu PC400',
 'Medium Truck',
 'Pump']

In [73]:
dd = iter(red_MTBF_this_week.index.tolist())
dd

In [74]:
PDTD_complete['fleet desc'].unique()

array(['Crane', 'Cat D8R', 'Cat D10', 'Komatsu D85ESS', 'Liebherr R984C',
       'Hitachi ZX350', 'Hitachi EX5500', 'Medium Truck', 'LV',
       'Cat 773E', 'Pump', 'Cat 785C OB truck', 'Komatsu HD785-7',
       'Cat 740', 'Cat 777D CoalTruck', 'Cat 745', 'Cat 777E',
       'Cat 777D OB truck', 'Manhaul', 'Genset', 'Air Compressor',
       'Compactor', 'Cat 834', 'Komatsu D65P-12', 'Driller',
       'Hitachi EX2500', 'Hitachi EX3600', 'Hitachi EX2600',
       'Komatsu PC400', 'Komatsu PC200', 'Hitachi ZX200', 'Komatsu PC300',
       'Cat 330GC', 'Cat 16M', 'Cat 18M', 'Cat 980 G/H', 'Cat 777D',
       'Mud Truck', 'Cat 773D', 'Komatsu HM400'], dtype=object)

Processing MTBF

In [75]:
#red_MTBF_this_week = complete_KPI['MTBF'][complete_KPI['MTBF']<100].dropna()
MTBF_df = PDTD_complete[PDTD_complete['fleet desc'].isin(red_MTBF_this_week.index)]
MTBF_df.to_excel('MTBF_df.xlsx')

In [76]:
'''MTBF Report template'''
MTBF_report = []


for i in dd:
    comp_gp_issue = []
    
    '''
    filter downtime detail for only failure event 
    '''
    
    na = MTBF_df[(MTBF_df['Is Failure']=='Yes')&(MTBF_df['fleet desc'] ==i)][['Reason','System']].groupby(['Reason'],as_index=False).count().sort_values('System',ascending=False)['Reason']
    count_of_failure = MTBF_df[(MTBF_df['fleet desc'] ==i)&(MTBF_df['Is Failure']=='Yes')].count()['Is Failure']
    '''
    conditional statement for unit that has no failure
    '''
    if count_of_failure == 0 or count_of_failure == 1:
        continue
    else:
        for u in na:    
            '''
            Create dataframe for system failure
            '''
            system_failure = MTBF_df[(MTBF_df['Is Failure']=='Yes')][['fleet desc','Reason','System','Part']].groupby(['fleet desc','Reason','System'],as_index=False).count()
            system_failure['MTBF_gp_repetitive'] = system_failure['System']+' '+system_failure['Part'].values.astype(str)+' times'
            
            '''
            Create dataframe for reason failure failure
            '''
            reason_failure = MTBF_df[(MTBF_df['fleet desc']==i) & (MTBF_df['Is Failure']=='Yes')]['Reason'].value_counts().reset_index()
            reason_failure['Reason_repetitive'] = reason_failure['index']+' '+reason_failure['Reason'].values.astype(str)+' times'

            general_comp_failure = []
            for e in reason_failure['index']:
                f = system_failure[(system_failure['fleet desc']==i)&(system_failure['Reason']==e)]['MTBF_gp_repetitive'].tolist()
                general_comp_failure.append(f)
            repetitive_issue = dict(zip(reason_failure['Reason_repetitive'],general_comp_failure))
        comp_gp_issue.append(repetitive_issue)
    '''
    report template per fleet model
    
    '''
    
    structured_MTBF = {
    'Fleet':i,
    'MTBF':red_MTBF_this_week[i],
    'Total Unit':All_KPI.groupby('fleet desc').count()['Unit'][i],
    'Total Hours Engine ON' : All_KPI.groupby('fleet desc').sum()['Total\nEngine On\nPeriod'][i],
    'Count of failures' : count_of_failure,
    'Failures by Comp Gp with repetitive issue' : comp_gp_issue 
    }
    
    MTBF_report.append(structured_MTBF)
    
MTBF_report

[{'Fleet': 'Cat 740',
  'MTBF': 56.09,
  'Total Unit': 24,
  'Total Hours Engine ON': 1660.8899999999999,
  'Count of failures': 31,
  'Failures by Comp Gp with repetitive issue': [{'011 - Engine 14 times': ['Bottom End 1 times',
     'Cooling 7 times',
     'Fuel 1 times',
     'Intake/Exhaust 5 times'],
    '019 - LV Electrical 4 times': ['Controlling 1 times',
     'Indicating 1 times',
     'Power 2 times'],
    '016 - Brakes 4 times': ['Hydraulic Delivery & Cooling 2 times',
     'Park Brake Assembly 2 times'],
    '012 - Torque Convertor/Transmission 3 times': ['Transmission 2 times'],
    '017 - Hydraulics 2 times': ['Hoist Delivery 1 times',
     'Steering Hydraulic Delivery & Cooling 1 times'],
    '018 - Chassis/Frame/Cabin/Steer Axles 2 times': ['Chassis 1 times',
     'Engine Enclosure/Engine Bay 1 times'],
    '023 - Aircon 1 times': ['Mechanical 1 times'],
    '013 - Drive Lines/Splitter Box/Swing/Drives 1 times': ['Transfer Gearbox 1 times']}]},
 {'Fleet': 'Cat 745',
  '

Processing MA

In [ ]:
red_MA_this_week = complete_KPI['MA'][complete_KPI['MA']<0.9].dropna()
MA_df= PDTD_complete.loc[PDTD_complete['fleet desc'].isin(red_MA_this_week.index)]


desc_MA= MA_df.loc[:,('Unit')]+' '+MA_df.loc[:,('Reported Fault/Job Description')]+' ('+MA_df.loc[:,('Period\nEvent\nMaint.\nDuration')].values.astype(str)+" Hours)"
red_MA_this_week


In [ ]:
MA_df.groupby(['Unit','Event\nID','Period\nEvent\nMaint.\nDuration'])['Reported Fault/Job Description'].apply(lambda x: ', '.join(x.astype(str))).reset_index()

In [ ]:
MA_df_new = MA_df.merge(desc_MA.rename('MA Anomali'),left_index=True, right_index=True).sort_values(['Period\nEvent\nMaint.\nDuration'],ascending=False)

In [ ]:
MA_df.columns

In [ ]:
MA_report_fail_df = MA_df.groupby(['Unit','Event\nID','Activity','fleet desc','Period\nEvent\nMaint.\nDuration'])['Reported Fault/Job Description'].apply(lambda x: ','.join(x.astype(str))).reset_index()
MA_report_fail_df['MA Anomali'] = MA_report_fail_df.loc[:,('Unit')]+' '+MA_report_fail_df.loc[:,('Reported Fault/Job Description')]+' ('+MA_report_fail_df.loc[:,('Period\nEvent\nMaint.\nDuration')].values.astype(str)+" Hours)"

MA_report_fail_df

In [ ]:
'''MA Report Template'''

MA_report = []

    
    
for i in red_MA_this_week.index:
    Scheduled_maintenance = []
    Unschedule_maintenance = []
    
    '''Processing MA df scheduled & unscheduled''' 
    
    MA_df_sched = MA_report_fail_df[((MA_report_fail_df.Activity =='08-Planned Maintenance (PMD)')&(MA_report_fail_df['fleet desc'] ==i)&(MA_report_fail_df['Period\nEvent\nMaint.\nDuration']>10.0))].sort_values(['Period\nEvent\nMaint.\nDuration'],ascending=False)
    MA_df_unsched = MA_report_fail_df[((MA_report_fail_df.Activity =='09-Unplanned Maintenance (UMD)')&(MA_report_fail_df['fleet desc'] ==i)&(MA_report_fail_df['Period\nEvent\nMaint.\nDuration']>10.0))].sort_values(['Period\nEvent\nMaint.\nDuration'],ascending=False)
    
    Scheduled_fleet_df = MA_df_sched['MA Anomali']
    Unscheduled_fleet_df = MA_df_unsched['MA Anomali'] 
    
    for x in Scheduled_fleet_df:
        Scheduled_maintenance.append(x)
    
    for x in Unscheduled_fleet_df:
        Unschedule_maintenance.append(x)
    
    
    structured_MA = {
        
    
    'Fleet':i,
    'MA':red_MA_this_week[i],
    'Total Unit':All_KPI.groupby('fleet desc').count()['Unit'][i],
    'UMD total hours':round(All_KPI.groupby('fleet desc').sum()['UMD\n(09)'][i],2),
    'PMD total hours':All_KPI.groupby('fleet desc').sum()['PMD\n(08)'][i],
    'PDAM total hours':All_KPI.groupby('fleet desc').sum()['PDAM\n08020\n09020'][i],
    'Scheduled maintenance' : Scheduled_maintenance,
    'Unscheduled maintenance' : Unschedule_maintenance
    
}
    MA_report.append(structured_MA)

MA_report


Processing MTTR

In [ ]:
red_MTTR_this_week = complete_KPI['MTTR'][complete_KPI['MTTR']>6].dropna()
MTTR_df = PDTD_complete[PDTD_complete['fleet desc'].isin(red_MTTR_this_week.index)]
#MTTR_df['Period\nEvent\nMaint.\nDuration'] = MTTR_df['Period\nEvent\nMaint.\nDuration']
#MTTR_df['MTTR Anomali']= MTTR_df['Unit']+' '+MTTR_df['Description of Repair']+' ('+MTTR_df['Period\nEvent\nMaint.\nDuration'].values.astype(str)+" Hours)"
red_MTTR_this_week

In [ ]:
MTTRF_report_fail_df = MTTR_df.groupby(['Unit','Event\nID','Activity','fleet desc','Period\nEvent\nMaint.\nDuration','Is Failure'])['Description of Repair'].apply(lambda x: ','.join(x.astype(str))).reset_index()
MTTRF_report_fail_df['MTTRF Anomali'] = MTTRF_report_fail_df.loc[:,('Unit')]+' '+MTTRF_report_fail_df.loc[:,('Description of Repair')]+' ('+MTTRF_report_fail_df.loc[:,('Period\nEvent\nMaint.\nDuration')].values.astype(str)+" Hours)"
MTTRF_report_fail_df.replace({'MTTRF Anomali':{'\n':' '}},regex=True)

In [ ]:
'''MTTR Report Template'''

MTTR_report = []
 
    
for i in red_MTTR_this_week.index:
    Breakdown_detail = []
    
    MTTR_breakdown_fail_yes =  MTTRF_report_fail_df[(MTTRF_report_fail_df['Is Failure'] =='Yes')&(MTTRF_report_fail_df['fleet desc'] ==i)&(MTTRF_report_fail_df['Period\nEvent\nMaint.\nDuration']>9.0)].sort_values(['Period\nEvent\nMaint.\nDuration'],ascending=False)
    
    for x in MTTR_breakdown_fail_yes['MTTRF Anomali']:
        Breakdown_detail.append(x)
        
    structured_MTTR = {
    'Fleet':i,
    'MTTR':red_MTTR_this_week[i],
    'Total Unit':All_KPI.groupby('fleet desc').count()['Unit'][i],
    'Hours Maintenance with Failure is Yes' : round(All_KPI[(All_KPI['fleet desc']==i)]['Failure Hours'].sum(),2),
    'Count of Failures' : MTTR_df[(MTTR_df['Is Failure'] =='Yes')&(MTTR_df['fleet desc'] ==i)].count()['Is Failure'],
    'Detail Breakdown with Failure is Yes' : Breakdown_detail
    
}
    MTTR_report.append(structured_MTTR)

In [ ]:
MTTR_report

In [ ]:
def set_col_widths(table):
    widths = (Mm(31.9), Mm(22.5), Mm(210.5))
    for row in table.rows:
        for idx, width in enumerate(widths):
            row.cells[idx].width = width

In [ ]:
'''repeatable header'''
def set_repeat_table_header(row):
    """ set repeat table row on every new page
    """
    tr = row._tr
    trPr = tr.get_or_add_trPr()
    tblHeader = OxmlElement('w:tblHeader')
    tblHeader.set(qn('w:val'), "true")
    trPr.append(tblHeader)
    return row

Write down to the word document

In [ ]:
'''Create word document using word'''
Anomali_report_full = Document()

section = Anomali_report_full.sections[0]
section.page_height = Mm(420)
section.page_width = Mm(297)
section.left_margin = Mm(25.4)
section.right_margin = Mm(10.5)
section.top_margin = Mm(22.2)
section.bottom_margin = Mm(10.5)
section.header_distance = Mm(12.7)
section.footer_distance = Mm(12.7)


'''shading elm with parse_xml'''

'''MA'''
shading_elm1 = parse_xml(r'<w:shd {} w:fill="66FFFF"/>'.format(nsdecls('w')))
shading_elm2 = parse_xml(r'<w:shd {} w:fill="66FFFF"/>'.format(nsdecls('w')))
shading_elm3 = parse_xml(r'<w:shd {} w:fill="66FFFF"/>'.format(nsdecls('w')))

'''MTBF'''
shading_elm4 = parse_xml(r'<w:shd {} w:fill="92D050"/>'.format(nsdecls('w')))
shading_elm5 = parse_xml(r'<w:shd {} w:fill="92D050"/>'.format(nsdecls('w')))
shading_elm6 = parse_xml(r'<w:shd {} w:fill="92D050"/>'.format(nsdecls('w')))


'''MTTR-F'''
shading_elm7 = parse_xml(r'<w:shd {} w:fill="FFC000"/>'.format(nsdecls('w')))
shading_elm8 = parse_xml(r'<w:shd {} w:fill="FFC000"/>'.format(nsdecls('w')))
shading_elm9 = parse_xml(r'<w:shd {} w:fill="FFC000"/>'.format(nsdecls('w')))




'''Mechanical Availability'''
Anomali_report_full.add_paragraph(text='Fleet with MA Below Target ').alignment = WD_ALIGN_PARAGRAPH.CENTER


'''
formatting MA table
'''

table_MA = Anomali_report_full.add_table(rows=1,cols=3)
table_MA.style = 'Table Grid'



hdr_cells_MA = table_MA.rows[0].cells
for cell in hdr_cells_MA:
    cell.height = Mm(30)
    
hdr_cells_MA[0].text = 'Fleet/Model'
hdr_cells_MA[1].text = f'MA week {nPeriod}'
hdr_cells_MA[2].text = 'Comments on Non-Compliance'

'''cells shading'''
hdr_cells_MA[0]._tc.get_or_add_tcPr().append(shading_elm1)
hdr_cells_MA[1]._tc.get_or_add_tcPr().append(shading_elm2)
hdr_cells_MA[2]._tc.get_or_add_tcPr().append(shading_elm3)

'''repeat header'''
set_repeat_table_header(table_MA.rows[0])


for i in MA_report:
    
    TOU_MA = i.get('Total Unit')
    PMD_MA = i.get('PMD total hours')
    UMD_MA = i.get('UMD total hours')
    PDAM_MA = i.get('PDAM total hours')
    
    row_cells_MA = table_MA.add_row().cells
    row_cells_MA[0].text = i.get('Fleet')
    row_cells_MA[1].text = "{:.2%}".format(i.get('MA'))
    
    sch = i.get('Scheduled maintenance')
    Unsch = i.get('Unscheduled maintenance')
    
    Scheduled_maintenance = '\n'.join([c for c in sch[1:]])
    Unscheduled_maintenance = '\n'.join([c for c in Unsch[1:]])
    
    non_compliance = f'Number of Units = {TOU_MA} \nPMD Total = {PMD_MA} \nUMD Total= {UMD_MA} \nPDAM Total hours= {PDAM_MA}\n'
    #MA_anomalies = f'Scheduled maintenance :\n{Scheduled_maintenance}\n\nUnscheduled maintenance :\n{Unscheduled_maintenance}'

    row_cells_MA[2].add_paragraph(text=non_compliance)
    row_cells_MA[2].add_paragraph('Scheduled maintenance :\n')
    for j in sch:
        row_cells_MA[2].add_paragraph(j,style='List Bullet')
    row_cells_MA[2].add_paragraph('\nUnscheduled maintenance :\n')
    for k in Unsch:
        row_cells_MA[2].add_paragraph(k,style='List Bullet')    

set_col_widths(table_MA)
Anomali_report_full.add_page_break()
    
'''
Mean Time Between Failure
'''
Anomali_report_full.add_paragraph(text='Fleet with MTBF Below Target ').alignment = WD_ALIGN_PARAGRAPH.CENTER

'''
formatting MTBF table 
'''

table_MTBF = Anomali_report_full.add_table(rows=1, cols=3)
table_MTBF.style = 'Table Grid'

hdr_cells_MTBF = table_MTBF.rows[0].cells
hdr_cells_MTBF[0].text = 'Fleet/Model '
hdr_cells_MTBF[1].text = f'MTBF week {nPeriod}'
hdr_cells_MTBF[2].text = 'Comments on Non-Compliance'

'''cells shading'''
hdr_cells_MTBF[0]._tc.get_or_add_tcPr().append(shading_elm4)
hdr_cells_MTBF[1]._tc.get_or_add_tcPr().append(shading_elm5)
hdr_cells_MTBF[2]._tc.get_or_add_tcPr().append(shading_elm6)

'''repeat header'''
set_repeat_table_header(table_MTBF.rows[0])


for d in MTBF_report:
    
    
    
    TOU = d.get('Total Unit')
    THEON = d.get('Total Hours Engine ON')
    COF = d.get('Count of failures')
    
    
    
    MTBF_value = str(d.get('MTBF'))
    row_cells = table_MTBF.add_row().cells
    row_cells[0].text = d.get('Fleet')
    row_cells[1].text = f'MTBF={MTBF_value}'
    
    gp_components = []
    
    
    for system in d.get('Failures by Comp Gp with repetitive issue'):
            for key, value in system.items():
                s = ", ".join(value)
                ter = f'{key}: {s}'
                gp_components.append(ter)
            
            
    f_list = '\n'.join([d for d in gp_components[0:]])
    fleet_data = f'Number of Units = {TOU} \nTotal Hours Engine On = {THEON} \n Count of failures = {COF} \n Failures by Comp Gp with repetitive issue : \n'
    
    row_cells[2].add_paragraph(fleet_data)
    for j in gp_components:
        row_cells[2].add_paragraph(j,style='List Bullet')


    
set_col_widths(table_MTBF)
Anomali_report_full.add_page_break()
    
'''
Mean Time to Repair Failure
'''
Anomali_report_full.add_paragraph(text='Fleet with MTTR-F Below Target ').alignment = WD_ALIGN_PARAGRAPH.CENTER

'''
formatting MTTRF table 
'''
table_MTTR = Anomali_report_full.add_table(rows=1, cols=3)
table_MTTR.style = 'Table Grid'

hdr_cells_MTTR = table_MTTR.rows[0].cells
hdr_cells_MTTR[0].text = 'Fleet/Model '
hdr_cells_MTTR[1].text = f'MTTR week {nPeriod}'
hdr_cells_MTTR[2].text = 'Comments on Non-Compliance'

'''cells shading'''
hdr_cells_MTTR[0]._tc.get_or_add_tcPr().append(shading_elm7)
hdr_cells_MTTR[1]._tc.get_or_add_tcPr().append(shading_elm8)
hdr_cells_MTTR[2]._tc.get_or_add_tcPr().append(shading_elm9)

'''repeat header'''
set_repeat_table_header(table_MTTR.rows[0])


for i in MTTR_report:
    
    TOU_MTTRF = i.get('MTTR')
    Hours_Fail_MTTRF = i.get('Hours Maintenance with Failure is Yes')
    COF_MTTRF = i.get('Count of Failures')
    DBF_MTTRF = i.get('Detail Breakdown with Failure is Yes')
    detail_breakdown_MTTRF = '\n\n'.join(map(str,DBF_MTTRF))
    
    #join([c for c in DBF_MTTRF[0:]])
        
    comment_MTTR = f'Number of Units = {TOU} \nHours maintenance with is Failure Yes ={Hours_Fail_MTTRF} \nCount of failures/Is Failure Yes = {COF_MTTRF} \n\n\n Detail breakdown with failure is yes:\n'
    
    MTTR_value = str(i.get('MTTR'))
    
    row_cells = table_MTTR.add_row().cells
    row_cells[0].text = i.get('Fleet')
    row_cells[1].text = f'MTTRF = {MTTR_value}'
    
    row_cells[2].add_paragraph(comment_MTTR)
    for j in DBF_MTTRF:
        row_cells[2].add_paragraph(j,style='List Bullet')
        

set_col_widths(table_MTTR)
Anomali_report_full.save('KPI Performance SGT Week 34.docx')
del Anomali_report_full

In [ ]:
# Will later be fixed :
# -Still choosing how the product will be used : software package or web

# Major :
# 1. Applied different standard of MTBF for different unit type (now all unit type has same standard >90)
# 2. Filter for only extreme hours.
# 3. MTBF value for AW unit that has low engine on hours but has no failure event. ---> tetap tercapai
# 4. MTTRF Bugs
#adjustable unit number for coal & OB


# Minor : 
# 1. Change MA format to %
# 2. Bullet list
# 3. Column width 
# 4. Column color
# 5. Page break

In [ ]:
fleet_desc_pivot1

In [ ]:
pareto_units = fleet_desc_pivot1[fleet_desc_pivot1['Critical Unit ?'].notna()]
pareto_units

In [ ]:
#pareto_units = pd.read_excel('Critical Unit.xlsx')
#pareto_units['fleet desc'].tolist()

In [ ]:
'''Processing Pareto'''
bad_Pareto_this_week = complete_KPI['MA'][complete_KPI['MA']<0.9].dropna()
Pareto_df = PDTD_complete[PDTD_complete['fleet desc'].isin(pareto_units['fleet desc'])]
red_Pareto_this_week=bad_Pareto_this_week[bad_Pareto_this_week.index.isin(pareto_units['fleet desc'].tolist())]
red_Pareto_this_week

In [ ]:
'''Pareto Report Template Column'''

Pareto_report_fail_df = Pareto_df.groupby(['Unit','Event\nID','Activity','fleet desc','Period\nEvent\nMaint.\nDuration','Is Failure'])['Description of Repair'].apply(lambda x: ','.join(x.astype(str))).reset_index()
Pareto_report_fail_df['Pareto Anomali'] = Pareto_report_fail_df.loc[:,('Unit')]+' '+Pareto_report_fail_df.loc[:,('Description of Repair')]+' ('+Pareto_report_fail_df.loc[:,('Period\nEvent\nMaint.\nDuration')].values.astype(str)+" Hours)"
Pareto_report_fail_df.replace({'Pareto Anomali':{'\n':' '}},regex=True)

In [ ]:
Pareto_report = []
 
    
for i in red_Pareto_this_week.index:
    Downtime_detail_sched = []
    Downtime_detail_unsched = []
    
    Pareto_df_sched = Pareto_report_fail_df[((Pareto_report_fail_df.Activity =='08-Planned Maintenance (PMD)')&(Pareto_report_fail_df['fleet desc'] ==i)&(Pareto_report_fail_df['Period\nEvent\nMaint.\nDuration']>1.0))].sort_values(['Period\nEvent\nMaint.\nDuration'],ascending=False)
    Pareto_df_unsched = Pareto_report_fail_df[((Pareto_report_fail_df.Activity =='09-Unplanned Maintenance (UMD)')&(Pareto_report_fail_df['fleet desc'] ==i)&(Pareto_report_fail_df['Period\nEvent\nMaint.\nDuration']>1.0))].sort_values(['Period\nEvent\nMaint.\nDuration'],ascending=False)
    

    for x in Pareto_df_sched['Pareto Anomali']:
        Downtime_detail_sched.append(x)
    
    for x in Pareto_df_unsched['Pareto Anomali']:
        Downtime_detail_unsched.append(x)

        
    structured_Pareto = {
    'Fleet':i,
    'Pareto':red_Pareto_this_week[i],
    'Total Unit':All_KPI.groupby('fleet desc').count()['Unit'][i],
    'UMD total hours':round(All_KPI.groupby('fleet desc').sum()['UMD\n(09)'][i],2),
    'PMD total hours':All_KPI.groupby('fleet desc').sum()['PMD\n(08)'][i],
    'PDAM total hours':All_KPI.groupby('fleet desc').sum()['PDAM\n08020\n09020'][i],    
        
    'Scheduled maintenance' : Downtime_detail_sched,
    'Unscheduled maintenance' : Downtime_detail_unsched
        
    
}
    Pareto_report.append(structured_Pareto)
Pareto_report

In [ ]:
'''Create word document using word'''
Pareto_report_full = Document()

section =Pareto_report_full.sections[0]
section.page_height = Mm(420)
section.page_width = Mm(297)
section.left_margin = Mm(25.4)
section.right_margin = Mm(10.5)
section.top_margin = Mm(22.2)
section.bottom_margin = Mm(10.5)
section.header_distance = Mm(12.7)
section.footer_distance = Mm(12.7)


'''Mechanical Availability'''


'''
formatting MA table
'''

table_Pareto = Pareto_report_full.add_table(rows=1,cols=3)
table_Pareto.style = 'Table Grid'


hdr_cells_Pareto = table_Pareto.rows[0].cells
for cell in hdr_cells_MA:
    cell.height = Mm(30)
    
hdr_cells_Pareto[0].text = 'Fleet/Model'
hdr_cells_Pareto[1].text = 'MA this week'
hdr_cells_Pareto[2].text = 'Comments on Non-Compliance'




for i in Pareto_report:
    
    TOU_MA = i.get('Total Unit')
    PMD_MA = i.get('PMD total hours')
    UMD_MA = i.get('UMD total hours')
    PDAM_MA = i.get('PDAM total hours')
    
    row_cells_MA = table_Pareto.add_row().cells
    row_cells_MA[0].text = i.get('Fleet')
    row_cells_MA[1].text = "{:.2%}".format(i.get('Pareto'))
    
    sch = i.get('Scheduled maintenance')
    Unsch = i.get('Unscheduled maintenance')
    
    Scheduled_maintenance = '\n'.join([c for c in sch[1:]])
    Unscheduled_maintenance = '\n'.join([c for c in Unsch[1:]])
    
    non_compliance = f'Number of Units = {TOU_MA} \nPMD Total = {PMD_MA} \nUMD Total= {UMD_MA} \nPDAM Total hours= {PDAM_MA}\n'
    #MA_anomalies = f'Scheduled maintenance :\n{Scheduled_maintenance}\n\nUnscheduled maintenance :\n{Unscheduled_maintenance}'

    row_cells_MA[2].add_paragraph(text=non_compliance)
    row_cells_MA[2].add_paragraph('Scheduled maintenance :\n')
    for j in sch:
        row_cells_MA[2].add_paragraph(j,style='List Bullet')
    row_cells_MA[2].add_paragraph('\nUnscheduled maintenance :\n')
    for k in Unsch:
        row_cells_MA[2].add_paragraph(k,style='List Bullet')    

set_col_widths(table_MA)
Pareto_report_full.add_page_break()

Pareto_report_full.save('Pareto report Week 34.docx')
del Pareto_report_full